### Tools

In [1]:
#%pip install -U langchain-community langgraph langchain-openai tavily-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 9.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 8.7 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.21
    Uninstalling langchain-core-0.2.21:
      Successfully uninstalled langchain-core-0.2.21
  Attempting uninstall: langgraph
    Found existing installation: langgraph 0.1.8
    Uninstalling langgraph-0.1.8:
      Successfully uninstalled langgraph-0.1.8
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.2.7
    Uninstalling langchain-community-0.2.7:
      Successfully uninstalled langchain-community-0.2.7
Note: you may need to restart the kernel to use updated packages.


In [17]:
import getpass
import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [18]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [19]:
from langchain_community.tools.tavily_search import TavilySearchResults

# define search tool

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in Los Angeles")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Los Angeles', 'region': 'California', 'country': 'United States of America', 'lat': 34.05, 'lon': -118.24, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1722452903, 'localtime': '2024-07-31 12:08'}, 'current': {'last_updated_epoch': 1722452400, 'last_updated': '2024-07-31 12:00', 'temp_c': 28.6, 'temp_f': 83.5, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 7.4, 'wind_kph': 11.9, 'wind_degree': 219, 'wind_dir': 'SW', 'pressure_mb': 1013.0, 'pressure_in': 29.92, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 33, 'cloud': 0, 'feelslike_c': 27.9, 'feelslike_f': 82.3, 'windchill_c': 28.6, 'windchill_f': 83.5, 'heatindex_c': 27.9, 'heatindex_f': 82.3, 'dewpoint_c': 10.9, 'dewpoint_f': 51.6, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 7.0, 'gust_mph': 8.5, 'gust_kph': 13.7}}"}, {'url': 'https://www.meteoprog.com/weather/Losangeles/mont

### Bind model with tools

In [5]:
#pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [20]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

# initialize model
model = ChatOpenAI(model="gpt-4o")

In [21]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [22]:
# bind tools with model
model_with_tools = model.bind_tools(tools)

In [23]:
# check response of model with search tool (no tool call)
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [24]:
# check response of model with search tool (tool call)
response = model_with_tools.invoke([HumanMessage(content="What's the weather in San Francisco?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_Z0V0RG2b4TQMeKWJLbOZLfnE', 'type': 'tool_call'}]


### Using a prebuilt react agent with tools

In [25]:
from langgraph.prebuilt import create_react_agent

#use prebuilt ReAct agent
agent_executor = create_react_agent(model, tools)

In [26]:
# test agent response (no tool call)
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='a63ec853-ce07-432c-886c-400be9526d87'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 81, 'total_tokens': 91}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_4e2b2da518', 'finish_reason': 'stop', 'logprobs': None}, id='run-4fd1a247-b739-49ab-841e-a61d45dc8f85-0', usage_metadata={'input_tokens': 81, 'output_tokens': 10, 'total_tokens': 91})]

In [27]:
# test agent response (tool call)
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="What's the weather in San Francisco?")]}
)
response["messages"]

[HumanMessage(content="What's the weather in San Francisco?", id='19d5a126-6b23-488a-8e71-acc65ff6805f'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Qh7Ietnbqa6Xlim8IggrD8fm', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 86, 'total_tokens': 108}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_bc2a86f5f5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6dde8f01-2141-43bd-bf69-686d042998b5-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_Qh7Ietnbqa6Xlim8IggrD8fm', 'type': 'tool_call'}], usage_metadata={'input_tokens': 86, 'output_tokens': 22, 'total_tokens': 108}),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'Cali